# 01 - Daft 介绍

## 学习目标

- 了解 Daft 是什么以及它的核心设计理念
- 理解 Daft 与其他 DataFrame 框架的区别
- 运行第一个 Daft 程序
- 理解延迟执行（Lazy Evaluation）的概念

## 1. 什么是 Daft？

Daft 是一个**分布式 DataFrame 框架**，专为大规模数据处理设计。它的核心特点：

- **延迟执行（Lazy Evaluation）**：操作不会立即执行，而是构建查询计划，在 `.collect()` 时才真正执行
- **查询优化**：自动优化执行路径，减少不必要的计算
- **分布式执行**：可在本地多核或通过 Ray 进行分布式计算
- **强类型系统**：支持基本类型、复杂类型（List、Struct）和时间类型

## 2. 为什么选择 Daft？

| 特性 | Pandas | Spark | Polars | Daft |
|------|--------|-------|--------|------|
| 执行模式 | 即时 | 延迟 | 延迟 | 延迟 |
| 分布式 | 否 | 是 | 否 | 是(Ray) |
| 查询优化 | 否 | 是 | 是 | 是 |
| Python UDF | 原生 | 受限 | 受限 | 原生 |
| 复杂类型 | 受限 | 支持 | 支持 | 支持 |
| 学习曲线 | 低 | 高 | 中 | 低 |

Daft 结合了 Pandas 的易用性和 Spark 的分布式能力，同时提供了现代化的查询优化。

## 3. 安装验证

In [ ]:
import daft
print(f"Daft 版本: {daft.__version__}")

In [ ]:
import sys
sys.path.insert(0, "..")

from src.examples import (
    read_products_csv,
    read_products_parquet,
    create_sample_dataframe,
    show_query_plan,
)

## 4. 第一个 Daft 程序

读取产品数据，查看 schema 和前几行。

In [ ]:
# 读取产品数据（推荐使用 Parquet 格式）
df = read_products_parquet()

# 查看 schema
print("Schema:")
print(df.schema())

In [ ]:
# 查看前几行数据
df.show(5)

In [ ]:
# 使用 create_sample_dataframe 获取 10 行示例数据
sample = create_sample_dataframe()
sample.show()

## 5. 延迟执行演示

Daft 的操作是延迟执行的。构建查询链时不会立即计算，直到调用 `.collect()` 或 `.show()` 才真正执行。

In [ ]:
from daft import col

# 构建查询链 —— 此时还没有真正执行
query = (
    df
    .where(col("price") > 1000)
    .select("product_id", "name", "category", "price")
    .sort("price", desc=True)
    .limit(10)
)

print("查询已构建，但尚未执行。")
print(f"类型: {type(query)}")

In [ ]:
# 查看查询计划
plan = show_query_plan(query)
print(plan)

In [ ]:
# 触发执行
result = query.collect()
print(f"结果行数: {len(result)}")
result.show()

## 6. 核心概念总结

| 概念 | 说明 |
|------|------|
| DataFrame | 分布式、延迟执行的数据结构 |
| Expression | 列操作表达式，如 `col("price") > 100` |
| 延迟执行 | 操作构建查询计划，`.collect()` 触发执行 |
| 查询优化 | 自动优化执行路径（谓词下推、列裁剪等） |
| Schema | 强类型列定义，支持复杂类型 |

## 下一步

继续学习 [02_basic_operations.ipynb](./02_basic_operations.ipynb) —— 掌握 Daft DataFrame 的基础操作。